In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
import sys
import json
from infer_functions import *


In [34]:
modules_to_reload = ["infer_functions"]
for module_name in modules_to_reload:
    if module_name in sys.modules:
        del sys.modules[module_name]
    __import__(module_name)
    module = sys.modules[module_name]
    globals().update({name: getattr(module, name) for name in dir(module) if not name.startswith('_')})

Data formation

In [17]:
static_tensor_dataset = create_tensor_dataset("static", 4, limit=100, categories= 0, averages=True)

Catting AAPL.csv
Catting ABBV.csv
Catting ABT.csv
Catting ACN.csv
Catting ADBE.csv
Catting AMAT.csv
Catting AMD.csv
Catting AMGN.csv
Catting AMZN.csv
Catting AVGO.csv
Catting BA.csv
Catting BKNG.csv
Catting CAT.csv
Catting CMCSA.csv
Catting COP.csv
Catting COST.csv
Catting CRM.csv
Catting CSCO.csv
Catting CVX.csv
Catting DHR.csv
Catting DIS.csv
Catting HD.csv
Catting HON.csv
Catting IBM.csv
Catting INTC.csv
Catting INTU.csv
Catting JNJ.csv
Catting KO.csv
Catting LIN.csv
Catting LLY.csv
Catting LOW.csv
Catting MA.csv
Catting MCD.csv
Catting MRK.csv
Catting MSFT.csv
Catting NKE.csv
Catting NOW.csv
Catting NVDA.csv
Catting ORCL.csv
Catting PEP.csv
Catting PFE.csv
Catting PG.csv
Catting PM.csv
Catting QCOM.csv
Catting RTX.csv
Catting SPGI.csv
Catting TMO.csv
Catting TMUS.csv
Catting TSLA.csv
Catting TXN.csv
Catting UBER.csv
Catting UNP.csv
Catting UPS.csv
Catting V.csv
Catting VZ.csv
Catting WMT.csv
Catting XOM.csv


In [33]:
dynamic_tensor_dataset = create_tensor_dataset("dynamic", 4, limit=100, categories= 0, averages=True)

Catting AAPL.csv
Catting ABBV.csv
Catting ABT.csv
Catting ACN.csv
Catting ADBE.csv
Catting AMAT.csv
Catting AMD.csv
Catting AMGN.csv
Catting AMZN.csv
Catting AVGO.csv
Catting BA.csv
Catting BKNG.csv
Catting BLK.csv
Catting BX.csv
Catting CAT.csv
Catting COP.csv
Catting CRM.csv
Catting CSCO.csv
Catting CVX.csv
Catting DHR.csv
Catting DIS.csv
Catting GE.csv
Catting HD.csv
Catting HON.csv
Catting IBM.csv
Catting INTC.csv
Catting INTU.csv
Catting JNJ.csv
Catting KO.csv
Catting LIN.csv
Catting LLY.csv
Catting LOW.csv
Catting MA.csv
Catting MCD.csv
Catting META.csv
Catting MRK.csv
Catting MSFT.csv
Catting NFLX.csv
Catting NOW.csv
Catting NVDA.csv
Catting ORCL.csv
Catting PEP.csv
Catting PFE.csv
Catting PG.csv
Catting PLD.csv
Catting PM.csv
Catting QCOM.csv
Catting RTX.csv
Catting SPGI.csv
Catting TMO.csv
Catting TMUS.csv
Catting TSLA.csv
Catting TXN.csv
Catting UBER.csv
Catting UNH.csv
Catting UNP.csv
Catting VZ.csv
Catting WMT.csv
tensor([[[1.5255e-05, 6.9777e-06, 1.1259e-05,  ..., 1.7972e-

In [19]:
print(len(dynamic_tensor_dataset))
for i, thing in list(enumerate(dynamic_tensor_dataset))[:100]:
    print(thing)

1977
((tensor([[1.5255e-05, 6.9777e-06, 1.1259e-05, 3.5437e-05, 1.7395e-05, 1.7972e-05,
         1.6365e-05, 2.3616e-05],
        [1.5761e-05, 7.0899e-06, 1.3775e-05, 6.9498e-05, 2.2771e-05, 2.5506e-05,
         1.8180e-05, 8.8955e-06],
        [2.0827e-05, 7.2160e-06, 1.7574e-05, 3.0669e-05, 3.3413e-05, 3.4662e-05,
         2.4180e-05, 4.4263e-05],
        [1.5593e-05, 4.6963e-06, 1.0889e-05, 1.3764e-05, 1.7267e-05, 1.6415e-05,
         1.6350e-05, 4.9416e-05]], dtype=torch.float64), tensor([0.0000e+00, 1.0000e+00, 4.0000e+00, 1.2730e+11], dtype=torch.float64)), tensor(0.2713, dtype=torch.float64))
((tensor([[1.7572e-05, 6.9777e-06, 1.9013e-05, 2.7140e-05, 3.5838e-05, 2.3573e-05,
         2.2302e-05, 3.0932e-05],
        [1.5537e-05, 7.0899e-06, 1.1481e-05, 3.6000e-05, 1.7275e-05, 1.8358e-05,
         1.6685e-05, 2.4297e-05],
        [1.6040e-05, 7.2160e-06, 1.4142e-05, 7.1264e-05, 2.4004e-05, 2.5966e-05,
         1.8489e-05, 9.1133e-06],
        [2.1213e-05, 7.3539e-06, 1.7919e-05, 3

In [20]:
with open(r"..\other_pickle\measures.json", "r") as file:
    measures = json.load(file)

static_size = len(measures["static"])
dynamic_size = len(measures["dynamic"])

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(device)

STATIC_BATCH_SIZE = 20
DYNAMIC_BATCH_SIZE = 5

static_train_dataloader = torch.utils.data.DataLoader(static_tensor_dataset, batch_size=STATIC_BATCH_SIZE, shuffle=True)
dynamic_train_dataloader = torch.utils.data.DataLoader(dynamic_tensor_dataset, batch_size=DYNAMIC_BATCH_SIZE, shuffle=False)

cpu


Static model

In [21]:
HIDDEN_SIZE = 200
LAYERS = 8

class StaticLSTM(nn.Module):

    def __init__(self, hidden_dim, batch_size, layers, input, categories=0):
        super(StaticLSTM, self).__init__()

        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.layers_num = layers
        
        #input is all the embedding vectors plus all the other variables
        self.lstm = nn.LSTM(input, hidden_dim, num_layers=layers, batch_first=True) 
        self.hidden = (torch.zeros(layers,batch_size,hidden_dim),torch.zeros(layers,batch_size,hidden_dim))
        
        #Squeeeze them into 1 dimension
        if categories > 0:
            self.hidden2label = nn.Linear(hidden_dim, categories)
        else:
            self.hidden2label = nn.Linear(hidden_dim, 1)

    def forward(self, batch_tensor):
        lstm_out, self.hidden = self.lstm(batch_tensor)
        last_timestep_output = lstm_out[:, -1, :]
        sales = self.hidden2label(last_timestep_output)
        return sales
    
    def hidden_reset(self):
        #reset the hidden and cell state after each epoch
        self.hidden = (torch.zeros(self.layers_num,self.batch_size,self.hidden_dim),
                       torch.zeros(self.layers_num,self.batch_size,self.hidden_dim))
    def batch_reset(self,batch_size):
        self.hidden = (torch.zeros(self.layers_num,batch_size,self.hidden_dim),
                       torch.zeros(self.layers_num,batch_size,self.hidden_dim))
    def flatten_parameters(self):
        self.lstm.flatten_parameters()

static_model = StaticLSTM(HIDDEN_SIZE, STATIC_BATCH_SIZE, LAYERS, static_size, categories=6)
static_model = static_model.double()

In [ ]:
loss_function = nn.L1Loss()
loss_scores = [float('inf')]
learning_rate = 0.01
epochs = 2
static_model = static_model.to(device)
optimizer = optim.Adam(static_model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    static_model.hidden_reset()
    epoch_loss = 0
    for i, batch in enumerate(static_train_dataloader):
        (input, worthless_input), label = batch
        if input.shape[0] != STATIC_BATCH_SIZE:
            static_model.batch_reset(input.shape[0])
            print("Reset triggered due to batch size mismatch")

        input, label = input.to(device), label.to(device)
        optimizer.zero_grad()
        output = static_model(input).squeeze()
        
        # Ensure output and label shapes are compatible for the loss function
        if output.shape != label.shape:
            print(f"Output shape: {output.shape}, Label shape: {label.shape}")
        
        loss = loss_function(output, label)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()

        if i % 50 == 49:
            print(f"Batch {i+1}, Loss: {epoch_loss / (i+1)}")
    
    average_epoch_loss = epoch_loss / len(static_train_dataloader)
    print(f"Average loss for epoch {epoch+1}: {average_epoch_loss}")
    
    if average_epoch_loss < loss_scores[-1]:
        torch.save(static_model.state_dict(), "../other_pickle/Static_Model.pth")
        print("Model saved")
    loss_scores.append(average_epoch_loss)

Static Grid Search

In [ ]:
x_train, y_train = split_tensor_dataset(static_tensor_dataset)
parameters = {
    'lr': [0.1, 0.01],  # Learning rate,  
    'max_epochs': [1, 2],
    "hidden_dim": [100, 200,300],
    "layers": [2,4,8]
}
Model, params = grid_search(static_model, parameters, x_train, y_train)

In [ ]:
static_model.load_state_dict(torch.load("../other_pickle/Static_Model.pth"))
static_model.eval()
for i, batch in list(enumerate(static_train_dataloader))[:10]:
    (input, worthless_input), label = batch
    output = static_model(input).squeeze()
    # loss = loss_function(output, label)
    print(output)

Dynamic model

In [23]:
HIDDEN_SIZE = 200
LAYERS = 8

class DynamicLSTM(nn.Module):

    def __init__(self, hidden_dim, batch_size, layers, input, categories=0):
        super(DynamicLSTM, self).__init__()

        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.layers_num = layers
        
        #input is all the embedding vectors plus all the other variables
        self.lstm = nn.LSTM(input, hidden_dim, num_layers=layers, batch_first=True) 
        self.hidden = (torch.zeros(layers,batch_size,hidden_dim),torch.zeros(layers,batch_size,hidden_dim))
        
        #Squeeeze them into 1 dimension
        if categories > 0:
            self.hidden2label = nn.Linear(hidden_dim, categories)
        else:
            self.hidden2label = nn.Linear(hidden_dim, 1)

    def forward(self, batch_tensor):
        lstm_out, self.hidden = self.lstm(batch_tensor)
        last_timestep_output = lstm_out[:, -1, :]
        sales = self.hidden2label(last_timestep_output)
        return sales
    
    def hidden_reset(self):
        #reset the hidden and cell state after each epoch
        self.hidden = (torch.zeros(self.layers_num,self.batch_size,self.hidden_dim),
                       torch.zeros(self.layers_num,self.batch_size,self.hidden_dim))
    def batch_reset(self,batch_size):
        self.hidden = (torch.zeros(self.layers_num,batch_size,self.hidden_dim),
                       torch.zeros(self.layers_num,batch_size,self.hidden_dim))
    def flatten_parameters(self):
        self.lstm.flatten_parameters()

dynamic_model = DynamicLSTM(HIDDEN_SIZE, STATIC_BATCH_SIZE, LAYERS, dynamic_size, categories=0)
dynamic_model = dynamic_model.double()

In [ ]:
# loss_function = nn.CrossEntropyLoss()
loss_function = nn.HuberLoss(delta=1.0)
loss_scores = [float('inf')]
learning_rate = 0.01
epochs = 1
dynamic_model = dynamic_model.to(device)
optimizer = optim.Adam(dynamic_model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    dynamic_model.hidden_reset()
    epoch_loss = 0
    for i, batch in enumerate(dynamic_train_dataloader):
        (input, worthless_input), label = batch
        if input.shape[0] != DYNAMIC_BATCH_SIZE:
            dynamic_model.batch_reset(input.shape[0])
            print("Reset triggered due to batch size mismatch")

        input, label = input.to(device), label.to(device)
        optimizer.zero_grad()
        output = dynamic_model(input).squeeze()
        
        # Ensure output and label shapes are compatible for the loss function
        if output.shape != label.shape:
            print(f"Output shape: {output.shape}, Label shape: {label.shape}")
        
        loss = loss_function(output, label)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()

        if i % 10 == 9:
            print(f"Batch {i+1}, Loss: {epoch_loss / (i+1)}")
    
    average_epoch_loss = epoch_loss / len(dynamic_train_dataloader)
    print(f"Average loss for epoch {epoch+1}: {average_epoch_loss}")
    
    if average_epoch_loss < loss_scores[-1]:
        torch.save(dynamic_model.state_dict(), "../other_pickle/Dynamic_Model.pth")
        print("Model saved")
    loss_scores.append(average_epoch_loss)

In [35]:
x_train, y_train = split_tensor_dataset(dynamic_tensor_dataset)
parameters = {
    'optimizer__lr': [0.1, 0.01],  # Learning rate,  
    'epochs': [1, 2],
    "module__hidden_dim": [100, 200,300],
    "module__layers": [2,4,8],
    'optimizer': [torch.optim.AdamW]
}
Model, params = grid_search(dynamic_model, parameters, x_train, y_train)

RuntimeError: Only Tensors created explicitly by the user (graph leaves) support the deepcopy protocol at the moment.  If you were attempting to deepcopy a module, this may be because of a torch.nn.utils.weight_norm usage, see https://github.com/pytorch/pytorch/pull/103001

In [30]:
dynamic_model.load_state_dict(torch.load("../other_pickle/Dynamic_Model.pth"))
dynamic_model.eval()
for i, batch in list(enumerate(dynamic_train_dataloader))[:100]:
    (input, worthless_input), label = batch
    output = dynamic_model(input).squeeze()
    # loss = loss_function(output, label)
    # print(F.softmax(output, dim=1), label)
    print(output, label)

tensor([0.0448, 0.0448, 0.0448, 0.0448, 0.0448], dtype=torch.float64,
       grad_fn=<SqueezeBackward0>) tensor([ 0.2713,  0.1489,  0.2096, -0.0097,  0.1655], dtype=torch.float64)
tensor([0.0448, 0.0448, 0.0448, 0.0448, 0.0448], dtype=torch.float64,
       grad_fn=<SqueezeBackward0>) tensor([ 0.0210,  0.1331, -0.1985, -0.0201,  0.1393], dtype=torch.float64)
tensor([0.0448, 0.0448, 0.0448, 0.0448, 0.0448], dtype=torch.float64,
       grad_fn=<SqueezeBackward0>) tensor([0.1282, 0.1938, 0.0883, 0.1082, 0.1084], dtype=torch.float64)
tensor([0.0448, 0.0448, 0.0448, 0.0448, 0.0448], dtype=torch.float64,
       grad_fn=<SqueezeBackward0>) tensor([ 0.0135, -0.1150, -0.0176, -0.0967, -0.0629], dtype=torch.float64)
tensor([0.0448, 0.0448, 0.0448, 0.0448, 0.0448], dtype=torch.float64,
       grad_fn=<SqueezeBackward0>) tensor([0.1300, 0.0443, 0.2106, 0.0685, 0.0525], dtype=torch.float64)
tensor([0.0448, 0.0448, 0.0448, 0.0448, 0.0448], dtype=torch.float64,
       grad_fn=<SqueezeBackward0>) tenso

Indicator model